In [14]:
import cv2
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

center_size=300

class CustomDataset(Dataset):
    def __init__(self, image_folder, label_folder, transform):
        self.image_folder = image_folder
        self.label_folder = label_folder
        self.transform = transform

        self.image_files = sorted(os.listdir(image_folder))
        self.label_files = sorted(os.listdir(label_folder))

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder, self.image_files[idx])
        label_name = os.path.join(self.label_folder, self.label_files[idx])

        img = Image.open(img_name)
        
        if self.transform:
            img = self.transform(img)

        with open(label_name, 'r') as label_file:
            label = label_file.read()
            label = label.split()
            x = float(label[0])
            y = float(label[1])

        return img, [x, y]
    
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=2, stride=2, padding=1)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=2, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=24, kernel_size=2, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(p=0.1)
        
        # 필터도 학습 가능하도록 설정
        self.conv1.weight.requires_grad = True
        self.conv2.weight.requires_grad = True
        self.conv3.weight.requires_grad = True
        
        self.fc1 = nn.Linear(24 * 38 * 38, 1000)
        self.fc2 = nn.Linear(1000,500)
        self.fc3 = nn.Linear(500, 250)
        self.fc4 = nn.Linear(250, 2)  # 2개의 출력을 가지는 모델

    def forward(self, x):
        x = self.conv1(x) # 151
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        
        x = x.view(-1, 24 * 38 * 38)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Grayscale(),  # 흑백 변환을 위해 추가
    transforms.ToTensor(),
    transforms.Resize((center_size, center_size)),
])

def cal(n=center_size, p=1, k=2, s=2):
    return ((n + 2 * p - k) / s + 1)

학습

In [3]:
image_path = "./images"
label_path = "./labels"

custom_dataset = CustomDataset(image_folder=image_path, label_folder=label_path, transform=transform)
data_loader = DataLoader(dataset=custom_dataset, shuffle=True, num_workers=0)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = SimpleCNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습
for epoch in range(100):
    for i, (images, labels) in enumerate(data_loader):
        images = images.float().to(device)
        labels = torch.tensor(labels, dtype=torch.float32).to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch+1}, Batch {i+1}, Loss: {loss.item()}")

torch.save(model.state_dict(), "face_center_net.pth")
print("학습 완료")

Epoch 1, Batch 1, Loss: 0.38699910044670105
Epoch 1, Batch 11, Loss: 0.3930409550666809
Epoch 1, Batch 21, Loss: 0.07079437375068665
Epoch 1, Batch 31, Loss: 0.03620481491088867
Epoch 1, Batch 41, Loss: 0.11088213324546814
Epoch 1, Batch 51, Loss: 0.22695671021938324
Epoch 1, Batch 61, Loss: 0.13416451215744019
Epoch 1, Batch 71, Loss: 0.02714022621512413
Epoch 1, Batch 81, Loss: 0.01139528863132
Epoch 1, Batch 91, Loss: 0.02171574905514717
Epoch 1, Batch 101, Loss: 0.16739948093891144
Epoch 1, Batch 111, Loss: 0.048940543085336685
Epoch 1, Batch 121, Loss: 0.01401511486619711
Epoch 1, Batch 131, Loss: 0.0021716072224080563
Epoch 1, Batch 141, Loss: 0.06169519200921059
Epoch 1, Batch 151, Loss: 0.04642932116985321
Epoch 1, Batch 161, Loss: 0.08072992414236069
Epoch 1, Batch 171, Loss: 0.00288220657967031
Epoch 1, Batch 181, Loss: 0.03998958319425583
Epoch 1, Batch 191, Loss: 0.05001077800989151
Epoch 1, Batch 201, Loss: 0.009250816889107227
Epoch 1, Batch 211, Loss: 0.03258907794952392

모델 결과 확인(train)

In [12]:
model = SimpleCNN()
model.load_state_dict(torch.load("face_center_net.pth"))
model.eval()

transform2 = transforms.Compose([
    transforms.Grayscale(),  # 흑백 변환을 위해 추가
    transforms.ToTensor(),
    transforms.Resize((center_size, center_size)),
])

pth = "./images"
lth = "./labels"

img_list = sorted(os.listdir("./images"))
lab_list = sorted(os.listdir("./labels"))

for i in range(len(img_list)):
    start_time = time.perf_counter()
    image_path = os.path.join(pth, img_list[i])
    label_path = os.path.join(lth, lab_list[i])
    print(img_list[i], lab_list[i])

    image = cv2.imread(image_path)
    image = cv2.resize(image, dsize=(center_size, center_size))

    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    with open(label_path, 'r') as label_file:
        label = label_file.read()
        label = list(map(float, label.split()))
        real_x, real_y = round(center_size * label[0]), round(center_size * label[1])

    # 이미지를 PyTorch의 Tensor로 변환
    imagein = Image.open(image_path)
    imagein = transform2(imagein)

    with torch.no_grad():
        output = model(imagein)
        model_output=output.squeeze().tolist()
        model_output=[round(x * center_size) for x in model_output]

    cv2.circle(image, (real_x, real_y), 5, (0, 255, 0), 2)
    
    cv2.circle(image, (model_output[0],model_output[1]), 5, (0, 0, 255), 2)

    cv2.imshow("model", image)
    end_time = time.perf_counter()
    elapsed_time = end_time - start_time

    print(f"실행 시간: {elapsed_time:.3f} 초")
    key = cv2.waitKey(0)
    if key == ord('q'):
        break
cv2.destroyAllWindows()

0-1 (1).jpg 0-1 (1).txt
실행 시간: 0.037 초


c:\Users\Administator\anaconda3\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


0-1 (10).jpg 0-1 (10).txt
실행 시간: 0.016 초
0-1 (100).jpg 0-1 (100).txt
실행 시간: 0.016 초
0-1 (1000).jpg 0-1 (1000).txt
실행 시간: 0.014 초
0-1 (1001).jpg 0-1 (1001).txt
실행 시간: 0.014 초
0-1 (1002).jpg 0-1 (1002).txt
실행 시간: 0.015 초
0-1 (1003).jpg 0-1 (1003).txt
실행 시간: 0.016 초
0-1 (1004).jpg 0-1 (1004).txt
실행 시간: 0.015 초
0-1 (1005).jpg 0-1 (1005).txt
실행 시간: 0.015 초
0-1 (1006).jpg 0-1 (1006).txt
실행 시간: 0.014 초
0-1 (1007).jpg 0-1 (1007).txt
실행 시간: 0.015 초
0-1 (1008).jpg 0-1 (1008).txt
실행 시간: 0.016 초
0-1 (1009).jpg 0-1 (1009).txt
실행 시간: 0.016 초
0-1 (101).jpg 0-1 (101).txt
실행 시간: 0.015 초
0-1 (1010).jpg 0-1 (1010).txt
실행 시간: 0.016 초
0-1 (1011).jpg 0-1 (1011).txt
실행 시간: 0.016 초
0-1 (1012).jpg 0-1 (1012).txt
실행 시간: 0.016 초
0-1 (1013).jpg 0-1 (1013).txt
실행 시간: 0.015 초
0-1 (1014).jpg 0-1 (1014).txt
실행 시간: 0.015 초
0-1 (1015).jpg 0-1 (1015).txt
실행 시간: 0.016 초
0-1 (1016).jpg 0-1 (1016).txt
실행 시간: 0.015 초
0-1 (1017).jpg 0-1 (1017).txt
실행 시간: 0.016 초
0-1 (1018).jpg 0-1 (1018).txt
실행 시간: 0.015 초
0-1 (1019).jpg 0-1

모델 결과 확인(video)

In [24]:
model = SimpleCNN()
model.load_state_dict(torch.load("face_center_net.pth"))
model.eval()

transform2 = transforms.Compose([
    transforms.Grayscale(),  # 흑백 변환을 위해 추가
    transforms.ToTensor(),
    transforms.Resize((center_size, center_size)),
])

readvideo=cv2.VideoCapture("center_screen_recording_alone.avi")
ret,image=readvideo.read()

while ret:
    start_time = time.perf_counter()
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    imagein = transform2(Image.fromarray(image))

    key = cv2.waitKey(1)

    with torch.no_grad():
        output = model(imagein)
        model_output=output.squeeze().tolist()
        model_output=[round(x * center_size) for x in model_output]
    
    cv2.circle(image, (model_output[0],model_output[1]), 5, (0, 0, 255), 2)
    cv2.imshow("win", image)
    
    end_time = time.perf_counter()
    elapsed_time = end_time - start_time
    print(f"실행 시간: {elapsed_time:.3f} 초")

    if key == ord("q"):
        break

    ret,image=readvideo.read()

readvideo.release()
cv2.destroyAllWindows()

실행 시간: 0.108 초
실행 시간: 0.036 초
실행 시간: 0.028 초
실행 시간: 0.021 초
실행 시간: 0.023 초
실행 시간: 0.026 초
실행 시간: 0.020 초
실행 시간: 0.020 초
실행 시간: 0.022 초
실행 시간: 0.025 초
실행 시간: 0.019 초
실행 시간: 0.019 초
실행 시간: 0.022 초
실행 시간: 0.018 초
실행 시간: 0.027 초
실행 시간: 0.031 초
실행 시간: 0.031 초
실행 시간: 0.029 초
실행 시간: 0.033 초
실행 시간: 0.032 초
실행 시간: 0.032 초
실행 시간: 0.027 초
실행 시간: 0.019 초
실행 시간: 0.024 초
실행 시간: 0.032 초
실행 시간: 0.030 초
실행 시간: 0.031 초
실행 시간: 0.028 초
실행 시간: 0.031 초
실행 시간: 0.031 초
실행 시간: 0.032 초
실행 시간: 0.030 초
실행 시간: 0.032 초
실행 시간: 0.029 초
실행 시간: 0.031 초
실행 시간: 0.033 초
실행 시간: 0.031 초
실행 시간: 0.027 초
실행 시간: 0.030 초
실행 시간: 0.031 초
실행 시간: 0.035 초
실행 시간: 0.018 초
실행 시간: 0.028 초
실행 시간: 0.029 초
실행 시간: 0.030 초
실행 시간: 0.031 초
실행 시간: 0.029 초
실행 시간: 0.032 초
실행 시간: 0.031 초
실행 시간: 0.031 초
실행 시간: 0.031 초
실행 시간: 0.030 초
실행 시간: 0.030 초
실행 시간: 0.032 초
실행 시간: 0.030 초
실행 시간: 0.030 초
실행 시간: 0.033 초
실행 시간: 0.029 초
실행 시간: 0.029 초
실행 시간: 0.032 초
실행 시간: 0.029 초
실행 시간: 0.024 초
실행 시간: 0.022 초
실행 시간: 0.031 초
실행 시간: 0.030 초
실행 시간: 0.019 초
실행 시간: 0.0

In [25]:
model = SimpleCNN()
model.load_state_dict(torch.load("face_center_net.pth"))
model.eval()

transform2 = transforms.Compose([
    transforms.Grayscale(),  # 흑백 변환을 위해 추가
    transforms.ToTensor(),
    transforms.Resize((center_size, center_size)),
])

readvideo=cv2.VideoCapture("center_screen_recording.avi")
ret,image=readvideo.read()

while ret:
    start_time = time.perf_counter()
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    imagein = transform2(Image.fromarray(image))

    key = cv2.waitKey(1)

    with torch.no_grad():
        output = model(imagein)
        model_output=output.squeeze().tolist()
        model_output=[round(x * center_size) for x in model_output]
    
    cv2.circle(image, (model_output[0],model_output[1]), 5, (0, 0, 255), 2)
    cv2.imshow("win", image)
    
    end_time = time.perf_counter()
    elapsed_time = end_time - start_time
    print(f"실행 시간: {elapsed_time:.3f} 초")

    if key == ord("q"):
        break

    ret,image=readvideo.read()

readvideo.release()
cv2.destroyAllWindows()

실행 시간: 0.034 초
실행 시간: 0.018 초
실행 시간: 0.016 초
실행 시간: 0.028 초
실행 시간: 0.015 초
실행 시간: 0.015 초
실행 시간: 0.016 초
실행 시간: 0.016 초
실행 시간: 0.016 초
실행 시간: 0.029 초
실행 시간: 0.016 초
실행 시간: 0.030 초
실행 시간: 0.016 초
실행 시간: 0.030 초
실행 시간: 0.029 초
실행 시간: 0.015 초
실행 시간: 0.013 초
실행 시간: 0.016 초
실행 시간: 0.016 초
실행 시간: 0.015 초
실행 시간: 0.015 초
실행 시간: 0.015 초
실행 시간: 0.016 초
실행 시간: 0.015 초
실행 시간: 0.015 초
실행 시간: 0.027 초
실행 시간: 0.017 초
실행 시간: 0.014 초
실행 시간: 0.016 초
실행 시간: 0.016 초
실행 시간: 0.017 초
실행 시간: 0.015 초
실행 시간: 0.016 초
실행 시간: 0.029 초
실행 시간: 0.015 초
실행 시간: 0.017 초
실행 시간: 0.033 초
실행 시간: 0.028 초
실행 시간: 0.027 초
실행 시간: 0.018 초
실행 시간: 0.031 초
실행 시간: 0.031 초
실행 시간: 0.029 초
실행 시간: 0.031 초
실행 시간: 0.033 초
실행 시간: 0.026 초
실행 시간: 0.029 초
실행 시간: 0.030 초
실행 시간: 0.032 초
실행 시간: 0.030 초
실행 시간: 0.029 초
실행 시간: 0.034 초
실행 시간: 0.030 초
실행 시간: 0.029 초
실행 시간: 0.029 초
실행 시간: 0.030 초
실행 시간: 0.018 초
실행 시간: 0.019 초
실행 시간: 0.022 초
실행 시간: 0.028 초
실행 시간: 0.032 초
실행 시간: 0.032 초
실행 시간: 0.019 초
실행 시간: 0.019 초
실행 시간: 0.021 초
실행 시간: 0.020 초
실행 시간: 0.0